# 불용어관련 패키기 Kiwi
######Kiwi =https://github.com/bab2min/kiwipiepy


In [ ]:
!pip3 install kiwipiepy

In [ ]:
'''
혹시 키위가 없다면..

!pip3 install --upgrade pip
!pip3 install kiwipiepy

나는 이같은 워닝 메시지가 나왔으나 그냥 일단 두고보기로 했어요
WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv

'''

In [ ]:
import pickle
import re
import numpy as np
import pandas as pd

## Google mount and set directory

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
directory = '/content/drive/MyDrive/00_projects/03_motherhood/00_recession/'

## Import file from drive

In [ ]:
chosun = pd.read_excel('/content/gdrive/MyDrive/00_projects/03_motherhood/00_recession/IMF_양육19680317_조선뉴스라이브러리.xlsx')

In [ ]:
chosun.dtypes

In [ ]:
#'sub2','sub3','sub4' 삭제
chosun = chosun.drop(['sub2','sub3','sub4'], axis = 1)

In [ ]:
chosun.head(10)

In [ ]:
chosun['sub1'][0]

In [ ]:
chosun['re_contents'][0]

In [ ]:
chosun.head()

## Contents Regulation

#### sub1 변수 정규화하여 특수문자, 다중공백 및 불필요한 문구 삭제

In [ ]:
"""
참조 https://acdongpgm.tistory.com/166
    https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221605317822

replace( ) 함수는 조건에 맞는 것을 다른 것으로 대체 해주는 함수.
r'[^\w]' 는 모든 특수문자를 뜻하는 정규 표현식.
df["variable"].str 을 하면 글자를 하나하나씩 다 쪼개준다.
repl = r'' 이건 제거하겠다는 의미 만약 r' '면 공백으로 대체됨.
regex=True 이거는 꼭 작성해줘야 정규표현식이 작동함

"""

# sub1 변수 내 contents re_contents변수로 새로 저장
  # 특수문자 및 숫자제거 -> r'' : 공백제거
  # 원본데이터가손상된경우글자가또는공란으로표기됩니다 삭제
chosun['re_contents'] = chosun['sub1'].str.replace(pat=r'[^A-Za-z-가-힣]', repl=r'', regex=True)
chosun['re_contents'] = chosun['re_contents'].str.replace("원본데이터가손상된경우글자가또는공란으로표기됩니다", "", regex=True)
chosun['re_contents'] = chosun['re_contents'].str.lower()

## Kiwi 품사태깅

In [ ]:
from kiwipiepy import Kiwi

In [ ]:
#  kiwi_chosun
kiwi_chosun = Kiwi()

In [ ]:
# 형태소분석 pos

pos_chosun = lambda x: kiwi_chosun.tokenize(x) if type (x) is str else None
chosun['pos_contents']  = chosun['re_contents'].apply(pos_chosun)

In [ ]:
from collections import Counter


'''주요 품사, 용언 품사 정의'''
주요품사 = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'SL']
용언품사 = ['VV', 'VA']


'''Counter를 활용해 가장 많이 나온 n개의 품사 결과를 돌려주는 pos_count() 함수'''
def pos_count(chosun, re_contents, output_filename, n=100):
    카운터 = Counter()

    for index, row in chosun.iterrows():
        if row[re_contents]:
            필터링결과 = [(token.form, token.tag) for token in row[re_contents] if token.tag in 주요품사]
            카운터.update(필터링결과)

    with open(output_filename, "w", encoding='utf-8-sig') as output_file:
        print("형태소,품사,개수", file=output_file)
        for (형태소, 품사), 개수 in 카운터.most_common(n):
            if 품사 in 용언품사:
                형태소 += "다"
            print(f"{형태소},{품사},{개수}", file=output_file)

In [ ]:
from gensim import corpora, models

'''주요 품사 정의'''
주요품사 = ["NNG", "NNP", "VV", "VA", "XR"]
용언품사 = ["VV", "VA" ]


'''형태소 분석 결과를 읽어서 주요 품사만 수집한 문서 리스트를 돌려준다.'''
def read_documents(df, col):
    문서리스트 = []
    for index, row in df.iterrows():
        if row[col]:
            필터링결과 = [(token.form, token.tag) for token in row[col] if token.tag in 주요품사]
            필터링결과 = [form+"다" if tag in 용언품사 else form for form, tag in 필터링결과]
        문서리스트.append(필터링결과)
    return 문서리스트


'''주어진 문서 집합으로 문서-어휘 행렬을 만들어 돌려준다.'''
def build_doc_term_mat(문서리스트):
    dictionary = corpora.Dictionary(문서리스트)
    corpus = [dictionary.doc2bow(문서) for 문서 in 문서리스트]
    return corpus, dictionary


'''문서-어휘 행렬을 TF-IDF 문서-단어 행렬로 변환한다.'''
def build_corpus_tfidf(corpus):
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf


'''토픽별 주요 단어들을 화면에 인쇄한다.'''
def print_topic_words(model):
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print(f"Topic ID: {topic_id}")
        for topic_word, prob in topic_word_probs:
            print(f"\t{topic_word}\t{prob:.3f}")
    print("\n")

In [ ]:
morph_analysis = lambda x: kiwi_chosun.tokenize(x) if type(x) is str else None
chosun['result'] = chosun['re_contents'].apply(morph_analysis)

# Topic modeling

#### 참조 https://hleecaster.com/python-kiwi-text-analysis-3/

In [ ]:
from gensim import corpora, models

'''주요 품사 정의'''
주요품사 = ["NNG", "NNP", "VV", "VA", "XR"]
용언품사 = ["VV", "VA" ]

